In [1]:
import numpy as np
import cv2

# image path
path = "test_direct/"
fileName = "78781177.jpg"

# Reading an image in default mode:
inputImage = cv2.imread(path + fileName)

# Convert the image to HSV:
frame_hsv = cv2.cvtColor(inputImage, cv2.COLOR_BGR2HSV)

# Prepare a dictionary to store the lower and upper
# HSV thresholds:
rangeDictionary = {}

# brown color
lower_values = np.array([6, 63, 0])
upper_values = np.array([23, 255, 81])

# push it into the dictionary:
rangeDictionary[0] = (lower_values, upper_values, "brown")

# gray color
lower_values = np.array([23, 0, 0])
upper_values = np.array([80, 105, 107])

# push it into the dictionary:
rangeDictionary[1] = (lower_values, upper_values, "gray")

# white color
lower_values = np.array([37, 0, 131])
upper_values = np.array([170, 25, 152])

# push it into the dictionary:
rangeDictionary[2] = (lower_values, upper_values, "white")

# Store results here:
targetRectangles = []

In [2]:
def areaFilter(minArea, inputImage):

    # Perform an area filter on the binary blobs:
    componentsNumber, labeledImage, componentStats, componentCentroids = \
    cv2.connectedComponentsWithStats(inputImage, connectivity=4)

    # Get the indices/labels of the remaining components based on the area stat
    # (skip the background component at index 0)
    remainingComponentLabels = [i for i in range(1, componentsNumber) if componentStats[i][4] >= minArea]

    # Filter the labeled pixels based on the remaining labels,
    # assign pixel intensity to 255 (uint8) for the remaining pixels
    filteredImage = np.where(np.isin(labeledImage, remainingComponentLabels) == True, 255, 0).astype('uint8')

    return filteredImage

In [5]:
for i in rangeDictionary:

    # Get current lower and upper range values:
    current_LowRange = rangeDictionary[i][0]
    current_UppRange = rangeDictionary[i][1]

    # Create the HSV mask
    mask = cv2.inRange(frame_hsv, current_LowRange, current_UppRange)

    # Run a minimum area filter:
    minArea = 800
    mask = areaFilter(minArea, mask)

     # Pre-process mask:
    kernelSize = 3

    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
    iterations = 10

    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, structuringElement, None, None, iterations, cv2.BORDER_REFLECT101)
    mask = cv2.morphologyEx(mask, cv2.MORPH_ERODE, structuringElement, None, None, iterations, cv2.BORDER_REFLECT101)

In [7]:
# Find the big contours/blobs on the filtered image:
contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

# List to store all the bounding rectangles:
contours_poly = [None] * len(contours)
boundRect = []

# Alright, just look for the outer bounding boxes:
for b, c in enumerate(contours):

    if hierarchy[0][b][3] == -1:

        # Approximate the contour to a polygon:
        contours_poly = cv2.approxPolyDP(c, 3, True)
        # Convert the polygon to a bounding rectangle:
        boundRect = cv2.boundingRect(contours_poly)

# Get the dimensions of the bounding rect:
        rectX = boundRect[0]
        rectY = boundRect[1]
        rectWidth = boundRect[2]
        rectHeight = boundRect[3]

        rectArea = rectWidth * rectHeight

        # Calculate the aspect ratio:
        aspectRatio = rectWidth / rectHeight
        delta = abs(1.0 - aspectRatio)

        # Set the min threshold values to identify the
        # blob of interest:
        minArea = 1000
        epsilon = 0.2

        # Is this bounding rectangle one the one we
        # are looking for?
        if rectArea > minArea and delta < epsilon:

            # Set a color:
            color = (0, 255, 0)
            inputCopy = inputImage.copy()

            # Draw the current rectangle on a copy of the BGR input:
            cv2.rectangle(inputCopy, (int(rectX), int(rectY)),
                            (int(rectX + rectWidth), int(rectY + rectHeight)), color, 2)
            # Store this bounding rectangle:
            targetRectangles.append(boundRect)


            # Label the current mask:
            currentColor = rangeDictionary[i][2]

            org = (rectX, rectY -10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            color = (255, 0, 0)
            cv2.putText(inputCopy, currentColor, org, font,
                        0.5, color, 1, cv2.LINE_AA)

            cv2.imwrite(path + "colorMask_"+currentColor+".png", inputCopy)

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import glob

for file in list: 
    img = cv.imread(file,0)
    ret2,th2 = cv.threshold(img,115,255,cv.THRESH_BINARY)
    # plt.subplot(3,3,4), plt.imshow(th2, 'gray')
    # plt.subplot(3,3,5), plt.imshow(th2, 'gray')

    fig, ax = plt.subplots(1, 2, figsize=(15,5))

    ax[0].imshow(th2)
    ax[1].imshow(img)
    plt.show()


In [ ]:
img = cv.imread(file,0)
ret2,th2 = cv.threshold(img,115,255,cv.THRESH_BINARY)
fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].imshow(th2)
ax[1].imshow(img)
plt.show()